## Capstone 1 - Part 2

In the first part of the capstone, we focused on Data Retrieval, Data Preprocessing, Feature Engineering and Exploratory Data Analysis using Python & its libraries. Now we are going to shift gears and gain insights into our HR Analytics dataset using SQL.

## TODO: Make use of SQL to do the following:

### Create a SQLITE3 DB using the CSV file (2 pts). Please refer this [link](https://www.linkedin.com/pulse/accessing-sqlite3-database-from-jupyter-notebook-using-varun-lobo/) and this [link](https://www.geeksforgeeks.org/how-to-import-csv-file-in-sqlite-database-using-python/) to know more.

### Calculate the Attrition Rate and summarize attrition (3 pts) by:
- Gender
- Department
- Age
- Average monthly income by job level
- Years at company

### Continue using SQL to explore main reasons for attrition (3 pts), For example:
- Why do more people over 50 years old leave the company than people who aged 40-50?
- Why do people with higher pay still leave the company?
- Which factors drive employees who work at company less than 5 years to leave?

### Effective Communication (2 pts)
- Please make use of markdown cells to communicate your thought process, why did you think of performing a step? what was the observation from the query? etc.
- The code should be commented so that it is readable for the reviewer.

### Grading and Important Instructions
- Each of the above steps are mandatory and should be completed in good faith
- Make sure before submitting that the code is in fully working condition
- It is fine to make use of ChatGPT, stackoverflow type resources, just provide the reference links from where you got it
- Debugging is an art, if you find yourself stuck with errors, take help of stackoverflow and ChatGPT to resolve the issue and if it's still unresolved, reach out to me for help.
- You need to score atleast 7/10 to pass the project, anything less than that will be marked required, needing resubmission.
- Feedback will be provided on 3 levels (Awesome, Suggestion, & Required). Required changes are mandatory to be made.
- For submission, please upload the project on github and share the link to the file with us through LMS.

#### Calculate Attrition Rate


In [10]:
### Implement code
import sqlite3
import pandas as pd

# Load the CSV file into a Pandas DataFrame
file_path = 'HR-Analytics.csv'  # Replace with the correct path if needed
df = pd.read_csv(file_path)

# Connect to SQLite3 and create a database
conn = sqlite3.connect('HR-Analytics.db')

# Save the DataFrame to SQLite
df.to_sql('hr_data', conn, if_exists='replace', index=False)
print("Database created and data loaded successfully.")

Database created and data loaded successfully.


#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857886?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4NiwiaG1hYyI6ImUyNGU0YWRhNWQwMDkxNWE2NWNiY2EwNGFlNDNiMTBlYmYyNWY0YzAiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Gender

In [14]:
### Implement code
# SQL query to calculate attrition by gender with true/false segregation
query_attrition_by_gender = '''
SELECT 
    ROW_NUMBER() OVER (ORDER BY Gender, Attrition) AS Row,
    CASE 
        WHEN Attrition = 'Yes' THEN 'true'
        ELSE 'false'
    END AS Attrition,
    Gender,
    COUNT(*) AS Count_Gender,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY Gender), 1) AS Attrition_by_Gender
FROM 
    hr_data
GROUP BY 
    Gender, Attrition
ORDER BY 
    Gender, Attrition;
'''

# Execute the query and fetch the result
attrition_by_gender = pd.read_sql_query(query_attrition_by_gender, conn)

# Rename columns for clarity
attrition_by_gender.rename(columns={
    'Row': 'Row',
    'Attrition': 'Attrition',
    'Gender': 'Gender',
    'Count_Gender': 'Count Gender',
    'Attrition_by_Gender': 'Attrition by Gender (%)'
}, inplace=True)

# Display the formatted DataFrame
print(attrition_by_gender)

   Row Attrition  Gender  Count Gender  Attrition by Gender (%)
0    1     false  Female           501                     85.2
1    2      true  Female            87                     14.8
2    3     false    Male           732                     83.0
3    4      true    Male           150                     17.0


The query counts the number of employees by gender and attrition status and calculates the attrition percentage for each gender using the formula:
COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY Gender) This gives the proportion of employees who left (attrition) out of the total employees for each gender.

#### Sample output

![image](https://api-v4.skyprepapp.com/public_api/da/857882?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4MiwiaG1hYyI6IjM1NDU5N2E4MjZmMTMxNWFhMTA4NzFiYTFkZjQ5YjBhNTYyOWYwMzgiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Dept

In [15]:
#### Find Attrition by Gender### Implement code
# SQL query to calculate attrition by department
query_department = '''
SELECT 
    Department,
    COUNT(*) AS Total_Employees,
    SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS Employees_Left,
    ROUND(SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS Attrition_Rate
FROM 
    hr_data
GROUP BY 
    Department;
'''

# Execute the query and display the results
department_attrition = pd.read_sql_query(query_department, conn)
print(department_attrition)


               Department  Total_Employees  Employees_Left  Attrition_Rate
0         Human Resources               63              12           19.05
1  Research & Development              961             133           13.84
2                   Sales              446              92           20.63


It does this by counting the total number of employees in each department and the number of employees who left (where Attrition = 'Yes'). The attrition rate is then calculated as the percentage of employees who left relative to the total employees in that department. The query groups the results by department to provide a department-wise breakdown. This helps in identifying departments with higher or lower turnover, which can be useful for HR decision-making.

#### Sample Output

<img src="https://api-v4.skyprepapp.com/public_api/da/857884?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4NCwiaG1hYyI6IjdmOTA4NzJjODZmYTgxOGUyMzkyYTBlZjhjYjljNjA4ZGM3NjkzMWYiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9" width="450" height="450">

#### Find Attrition by Age Groups

In [17]:
### Implement code
# SQL query to calculate attrition by age groups with true/false segregation
query_attrition_by_age_groups = '''
SELECT 
    ROW_NUMBER() OVER (ORDER BY Age_Group, Attrition) AS Row,
    CASE 
        WHEN Attrition = 'Yes' THEN 'true'
        ELSE 'false'
    END AS Attrition,
    Age_Group,
    COUNT(*) AS Count_Age_Group,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY Age_Group), 1) AS Attrition_by_Age_Group
FROM 
    (SELECT 
        *,
        CASE 
            WHEN Age < 30 THEN '<30'
            WHEN Age BETWEEN 30 AND 39 THEN '30-39'
            WHEN Age BETWEEN 40 AND 49 THEN '40-49'
            ELSE '>50'
        END AS Age_Group
    FROM 
        hr_data) AS Age_Categorized
GROUP BY 
    Age_Group, Attrition
ORDER BY 
    Age_Group, Attrition;
'''

# Execute the query and fetch the result
attrition_by_age_groups = pd.read_sql_query(query_attrition_by_age_groups, conn)

# Rename columns for clarity
attrition_by_age_groups.rename(columns={
    'Row': 'Row',
    'Attrition': 'Attrition',
    'Age_Group': 'Age Group',
    'Count_Age_Group': 'Count Age Group',
    'Attrition_by_Age_Group': 'Attrition by Age Group (%)'
}, inplace=True)

# Display the formatted DataFrame
print(attrition_by_age_groups)


   Row Attrition Age Group  Count Age Group  Attrition by Age Group (%)
0    1     false     30-39              533                        85.7
1    2      true     30-39               89                        14.3
2    3     false     40-49              315                        90.3
3    4      true     40-49               34                         9.7
4    5     false       <30              235                        72.1
5    6      true       <30               91                        27.9
6    7     false       >50              150                        86.7
7    8      true       >50               23                        13.3


#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857885?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4NSwiaG1hYyI6IjcwNGVlOWIwYzg1MmMwNmNhODg0NmYyZDFlNWE3OTU1MTFhMGVmYWYiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Monthly Income

In [19]:
### Implement code
# Updated SQL query
query_attrition_by_income = '''
SELECT 
    ROW_NUMBER() OVER (ORDER BY Department, JobLevel, AVG(MonthlyIncome)) AS Row,
    Department,
    JobLevel,
    ROUND(AVG(CASE WHEN Attrition = 'Yes' THEN MonthlyIncome ELSE NULL END), 2) AS Attrition_Avg_Income,
    ROUND(AVG(CASE WHEN Attrition = 'No' THEN MonthlyIncome ELSE NULL END), 2) AS Non_Attrition_Avg_Income,
    ROUND(AVG(CASE WHEN Attrition = 'Yes' THEN MonthlyIncome ELSE NULL END) -
          AVG(CASE WHEN Attrition = 'No' THEN MonthlyIncome ELSE NULL END), 2) AS Income_Difference
FROM 
    hr_data
GROUP BY 
    Department, JobLevel
ORDER BY 
    Department, JobLevel, AVG(MonthlyIncome);
'''

# Execute the query and fetch the result
attrition_by_income = pd.read_sql_query(query_attrition_by_income, conn)

# Rename columns for clarity
attrition_by_income.rename(columns={
    'Row': 'Row',
    'Department': 'Department',
    'JobLevel': 'Job Level',
    'Attrition_Avg_Income': 'Attrition Average Income',
    'Non_Attrition_Avg_Income': 'Non-Attrition Average Income',
    'Income_Difference': 'Income Difference'
}, inplace=True)

# Display the formatted DataFrame
print(attrition_by_income)


    Row              Department  Job Level  Attrition Average Income  \
0     1         Human Resources          1                   2415.70   
1     2         Human Resources          2                       NaN   
2     3         Human Resources          3                  10216.00   
3     4         Human Resources          4                       NaN   
4     5         Human Resources          5                       NaN   
5     6  Research & Development          1                   2687.38   
6     7  Research & Development          2                   5372.00   
7     8  Research & Development          3                   9503.85   
8     9  Research & Development          4                  12169.00   
9    10  Research & Development          5                  19550.00   
10   11                   Sales          1                   2373.44   
11   12                   Sales          2                   5917.00   
12   13                   Sales          3                   920

The Query compares the average income of employees who left (attrition) and those who stayed, broken down by department and job level. It calculates the average income for both groups and the difference between them, helping identify if income levels play a role in employee turnover.

#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857883?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4MywiaG1hYyI6Ijg3NTU4ZDU1ZjRjN2U1YWI3ODQzYjM1NzFkNjBjMjEwNGY5NWI5ODUiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Find Attrition by Years At Company

In [21]:
### Implement code
# SQL query to group attrition by years at company into specific ranges
query_attrition_by_tenure = '''
WITH TenureGroups AS (
    SELECT 
        CASE 
            WHEN YearsAtCompany = 0 THEN 'New Hires'
            WHEN YearsAtCompany BETWEEN 2 AND 5 THEN '2-5 Years'
            WHEN YearsAtCompany BETWEEN 6 AND 10 THEN '6-10 Years'
            WHEN YearsAtCompany BETWEEN 11 AND 20 THEN '11-20 Years'
            WHEN YearsAtCompany > 20 THEN 'Over 20 Years'
            ELSE 'Other'
        END AS Tenure_Group,
        Attrition
    FROM 
        hr_data
)
SELECT 
    ROW_NUMBER() OVER (ORDER BY Tenure_Group) AS Row,
    Tenure_Group AS Tenure_Years,
    COUNT(CASE WHEN Attrition = 'Yes' THEN 1 ELSE NULL END) AS Num,
    ROUND(COUNT(CASE WHEN Attrition = 'Yes' THEN 1 ELSE NULL END) * 100.0 / COUNT(*), 2) AS Percent
FROM 
    TenureGroups
GROUP BY 
    Tenure_Group
ORDER BY 
    ROW;
'''

# Execute the query and fetch the result
attrition_by_tenure = pd.read_sql_query(query_attrition_by_tenure, conn)

# Rename columns for clarity
attrition_by_tenure.rename(columns={
    'Row': 'Row',
    'Tenure_Years': 'Tenure Years',
    'Num': 'Num (Attrition Count)',
    'Percent': 'Attrition Percent (%)'
}, inplace=True)

# Display the formatted DataFrame
print(attrition_by_tenure)


   Row   Tenure Years  Num (Attrition Count)  Attrition Percent (%)
0    1    11-20 Years                     12                   6.67
1    2      2-5 Years                     87                  15.51
2    3     6-10 Years                     55                  12.28
3    4      New Hires                     16                  36.36
4    5          Other                     59                  34.50
5    6  Over 20 Years                      8                  12.12


This SQL query categorizes employees based on their years at the company into specific tenure groups (e.g., "New Hires," "2-5 Years,") and calculates attrition for each group. It counts the number of employees who left (attrition) and computes the attrition percentage within each tenure group. This helps identify if employees in certain tenure ranges have higher turnover, offering insights into retention strategies for different experience levels.

#### Sample Output

![image](https://api-v4.skyprepapp.com/public_api/da/857881?view_key=eyJvYmplY3RfdHlwZSI6ImFydGljbGUiLCJvYmplY3RfaWQiOjg1Nzg4MSwiaG1hYyI6IjFhYmY3NGI4MzQ1NzViMWZkNDJlMjcwYTUyOTQ0OWQwZjJjMjhmNWUiLCJleHBpcmVzX2F0Ijo0ODkxMzgxMTk5LCJvdHRfdG9rZW4iOm51bGx9)

#### Continue using SQL to explore main reasons for attrition (3 pts), For example:
- Why do more people over 50 years old leave the company than people who aged 40-50?
- Why do people with higher pay still leave the company?
- Which factors drive employees who work at company less than 5 years to leave?

In [22]:
# SQL query to compare attrition rates for employees over 50 vs 40-50
query_over_50 = '''
SELECT 
    CASE 
        WHEN Age > 50 THEN 'Over 50'
        WHEN Age BETWEEN 40 AND 50 THEN '40-50'
    END AS Age_Group,
    COUNT(*) AS Total_Employees,
    SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS Employees_Left,
    ROUND(SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS Attrition_Rate
FROM 
    hr_data
WHERE Age >= 40
GROUP BY 
    Age_Group;
'''

over_50_attrition = pd.read_sql_query(query_over_50, conn)
print(over_50_attrition)


  Age_Group  Total_Employees  Employees_Left  Attrition_Rate
0     40-50              379              39           10.29
1   Over 50              143              18           12.59


This indicates that employees over 50 have a higher attrition rate (12.59%) compared to those aged 40-50 (10.29%).

Possible Reasons could be-
Retirement,health issues, economic factors

In [23]:
# SQL query to analyze attrition by monthly income
query_income = '''
SELECT 
    JobLevel,
    AVG(MonthlyIncome) AS Average_Income,
    SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS Employees_Left,
    ROUND(SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS Attrition_Rate
FROM 
    hr_data
GROUP BY 
    JobLevel;
'''

income_attrition = pd.read_sql_query(query_income, conn)
print(income_attrition)


   JobLevel  Average_Income  Employees_Left  Attrition_Rate
0         1     2786.915285             143           26.34
1         2     5502.277154              52            9.74
2         3     9817.252294              32           14.68
3         4    15503.783019               5            4.72
4         5    19191.826087               5            7.25


The query and its output suggest that employees at lower job levels (e.g., JobLevel 1) with lower average monthly incomes exhibit a higher attrition rate (26.34%) compared to those at higher job levels (e.g., JobLevel 5 with an attrition rate of 7.25%).

However, even employees with higher pay (e.g., JobLevel 5) still leave the company, as evidenced by their attrition rates of 7.25% and 4.72%.

Possible Reasons - Job Satisfaction and Career Growth, Better opportunities

In [24]:
# SQL query to analyze attrition for employees with less than 5 years at the company
query_years = '''
SELECT 
    CASE 
        WHEN YearsAtCompany < 5 THEN 'Less than 5 years'
        ELSE '5 years or more'
    END AS Experience_Group,
    COUNT(*) AS Total_Employees,
    SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) AS Employees_Left,
    ROUND(SUM(CASE WHEN Attrition = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS Attrition_Rate
FROM 
    hr_data
GROUP BY 
    Experience_Group;
'''

years_attrition = pd.read_sql_query(query_years, conn)
print(years_attrition)


    Experience_Group  Total_Employees  Employees_Left  Attrition_Rate
0    5 years or more              890              96           10.79
1  Less than 5 years              580             141           24.31


Employees who have been with the company for less than 5 years may leave due to factors such as lack of career growth, job mismatch, poor work-life balance, inadequate compensation, or dissatisfaction with company culture.






